# Documentation for processed Census data

This notebook walks through the steps to arrive at the processed Census data used in `zipbiaschecker`.

## Sources of original reference data

One challenge of mapping zip codes to demographic data is that one accessible source of demographic data from Census surveys is reported on the census tract, not zip code, level. To map zip code to demographic data, we then need to map zip code first to census tract. The source of this zip-to-tract mapping comes from the US Department of Housing and Urban Development's [crosswalk files](https://www.huduser.gov/portal/datasets/usps_crosswalk.html#data). This tool uses the dataset extracted when selecting "ZIP-TRACT" for crosswalk type and "1st Quarter 2020" for time period (see picture below).

![HUD data source](../images/HUD_zip_to_tract_data.png)

Next, we pull the most recent demographic estimates for each census tract from [the American Community Survey's 5-year view from 2018](https://data.census.gov/cedsci/table?q=DP05%3A%20ACS%20DEMOGRAPHIC%20AND%20HOUSING%20ESTIMATES&hidePreview=true&tid=ACSDP5Y2018.DP05&g=0400000US17.140000,01.140000,02.140000,04.140000,05.140000,06.140000,08.140000,09.140000,10.140000,11.140000,12.140000,13.140000,15.140000,16.140000,18.140000,19.140000,20.140000,21.140000,22.140000,23.140000,24.140000,25.140000,26.140000,27.140000,28.140000,29.140000,30.140000,31.140000,32.140000,33.140000,34.140000,35.140000,36.140000,37.140000,38.140000,39.140000,40.140000,41.140000,42.140000,44.140000,45.140000,46.140000,47.140000,48.140000,49.140000,50.140000,51.140000,53.140000,54.140000,55.140000,56.140000). This link should have data for the 50 states and DC pre-selected, displayed as such:

![ACS data source](../images/ACS_demographic_data.png)

The original data as of 6/30/2020 has been downloaded and saved in this code base. We take a quick look at the HUD and ACS data below.

In [1]:
import pandas as pd
pd.set_option('max_colwidth', 999)

hud_ct_zip_df = pd.read_excel('../data/original/ZIP_TRACT_032020.xlsx')
hud_ct_zip_df.TRACT = [x.zfill(11) for x in hud_ct_zip_df.TRACT.astype(str)]
hud_ct_zip_df.ZIP = [x.zfill(5) for x in hud_ct_zip_df.ZIP.astype(str)]
print(hud_ct_zip_df.shape)
print('Number of unique census tracts:', hud_ct_zip_df.TRACT.nunique())
print('Number of unique zip codes:', hud_ct_zip_df.ZIP.nunique())
hud_ct_zip_df.head()

(172121, 6)
Number of unique census tracts: 73467
Number of unique zip codes: 39325


,ZIP,TRACT,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,00501,36103158607,0.000000,1.000000,0.000000,1.000000
1,00601,72001956800,0.014816,0.012563,0.044025,0.015412
2,00601,72113071700,0.161060,0.206030,0.132075,0.163171
3,00601,72001956600,0.172913,0.354271,0.358491,0.189069
4,00601,72001956700,0.651211,0.427136,0.465409,0.632348


The `RES_RATIO` column will be useful later when allocating data from zip tracts that span multiple zip codes appropriately.

In [2]:
acs_ct_demo_df = pd.read_csv('../data/original/ACSDP5Y2018.DP05_2020-06-30T134004/ACSDP5Y2018.DP05_data_with_overlays_2020-06-30T133432.csv')
print(acs_ct_demo_df.shape)
acs_ct_demo_df.head()

//anaconda3/envs/python3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7,8,9,11,12,15,16,19,20,23,24,27,28,31,32,35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64,67,68,71,72,75,76,79,80,83,84,87,88,91,92,95,96,99,100,103,104,107,108,111,112,115,116,119,120,123,124,127,128,129,131,132,135,136,139,140,143,144,147,148,151,152,155,156,157,159,160,163,164,167,168,171,172,175,176,179,180,183,184,187,188,191,192,195,196,199,200,203,204,207,208,211,212,215,216,219,220,223,224,225,227,228,231,232,235,236,237,239,240,243,244,251,252,255,256,259,260,263,264,267,268,271,272,275,276,279,280,283,284,287,288,291,292,295,296,299,300,307,308,311,312,315,316,319,320,323,324,327,328,331,332,333,335,336,339,340,347,348,349,351,352,355,356) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(73057, 358)


,GEO_ID,NAME,DP05_0031PM,DP05_0032E,DP05_0032M,DP05_0032PE,DP05_0032PM,DP05_0033E,DP05_0033M,DP05_0033PE,...,DP05_0029M,DP05_0029PE,DP05_0029PM,DP05_0030E,DP05_0030M,DP05_0030PE,DP05_0030PM,DP05_0031E,DP05_0031M,DP05_0031PE
0,id,Geographic Area Name,Percent Margin of Error!!SEX AND AGE!!Total population!!65 years and over!!Female,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females),Margin of Error!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females),Percent Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females),Percent Margin of Error!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females),Estimate!!RACE!!Total population,Margin of Error!!RACE!!Total population,Percent Estimate!!RACE!!Total population,...,Margin of Error!!SEX AND AGE!!Total population!!65 years and over,Percent Estimate!!SEX AND AGE!!Total population!!65 years and over,Percent Margin of Error!!SEX AND AGE!!Total population!!65 years and over,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Male,Margin of Error!!SEX AND AGE!!Total population!!65 years and over!!Male,Percent Estimate!!SEX AND AGE!!Total population!!65 years and over!!Male,Percent Margin of Error!!SEX AND AGE!!Total population!!65 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Female,Margin of Error!!SEX AND AGE!!Total population!!65 years and over!!Female,Percent Estimate!!SEX AND AGE!!Total population!!65 years and over!!Female
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",13.4,75.0,42.8,(X),(X),1923,253,1923,...,52,245,(X),105,43,42.9,13.4,140,40,57.1
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",11.9,36.5,22.8,(X),(X),2028,192,2028,...,79,284,(X),76,38,26.8,11.9,208,70,73.2
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",8.8,66.2,24.1,(X),(X),3476,433,3476,...,87,580,(X),231,58,39.8,8.8,349,76,60.2
4,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",5.3,86.9,18.6,(X),(X),3831,337,3831,...,119,901,(X),419,75,46.5,5.3,482,78,53.5


## Methodology to process final reference data

This section will show the logic to process the final reference data.

In [3]:
# Take specific columns from the ACS data
acs_ct_demo_df['census_tract'] = [x[-11:] for x in acs_ct_demo_df.GEO_ID]
acs_ct_demo_df = acs_ct_demo_df[['census_tract', 'DP05_0063E', 'DP05_0065E', 'DP05_0066E', 'DP05_0070E', 'DP05_0071E']]
acs_ct_demo_df.head()

,census_tract,DP05_0063E,DP05_0065E,DP05_0066E,DP05_0070E,DP05_0071E
0,id,Estimate!!Race alone or in combination with one or more other races!!Total population,Estimate!!Race alone or in combination with one or more other races!!Total population!!Black or African American,Estimate!!Race alone or in combination with one or more other races!!Total population!!American Indian and Alaska Native,Estimate!!HISPANIC OR LATINO AND RACE!!Total population,Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)
1,01001020100,1923,227,46,1923,68
2,01001020200,2028,1129,25,2028,36
3,01001020300,3476,775,20,3476,299
4,01001020400,3831,341,61,3831,90


In [4]:
acs_ct_demo_df = acs_ct_demo_df.iloc[1:, :]
acs_ct_demo_df.head()

,census_tract,DP05_0063E,DP05_0065E,DP05_0066E,DP05_0070E,DP05_0071E
1,01001020100,1923,227,46,1923,68
2,01001020200,2028,1129,25,2028,36
3,01001020300,3476,775,20,3476,299
4,01001020400,3831,341,61,3831,90
5,01001020500,9883,1746,0,9883,122


In [5]:
acs_ct_demo_df['percent_black'] = acs_ct_demo_df['DP05_0065E'].astype(int) / acs_ct_demo_df['DP05_0063E'].astype(int)
acs_ct_demo_df['percent_indigenous'] = acs_ct_demo_df['DP05_0066E'].astype(int) / acs_ct_demo_df['DP05_0063E'].astype(int)
acs_ct_demo_df['percent_hispanic'] = acs_ct_demo_df['DP05_0071E'].astype(int) / acs_ct_demo_df['DP05_0070E'].astype(int)
acs_ct_demo_df = acs_ct_demo_df[['census_tract', 'percent_black', 'percent_indigenous', 'percent_hispanic']]
print(acs_ct_demo_df.shape)
acs_ct_demo_df.head()

(73056, 4)


,census_tract,percent_black,percent_indigenous,percent_hispanic
1,01001020100,0.118045,0.023921,0.035361
2,01001020200,0.556706,0.012327,0.017751
3,01001020300,0.222957,0.005754,0.086018
4,01001020400,0.089011,0.015923,0.023493
5,01001020500,0.176667,0.000000,0.012344


In [6]:
hud_ct_zip_df = hud_ct_zip_df.rename({'TRACT':'census_tract'}, axis = 1)
hud_ct_zip_df = pd.merge(hud_ct_zip_df, acs_ct_demo_df, how='inner', on = 'census_tract')
print(hud_ct_zip_df.shape)
print('Number of unique census tracts:', hud_ct_zip_df.census_tract.nunique())
print('Number of unique zip codes:', hud_ct_zip_df.ZIP.nunique())
hud_ct_zip_df.tail()

(170531, 9)
Number of unique census tracts: 72616
Number of unique zip codes: 39125


,ZIP,census_tract,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,percent_black,percent_indigenous,percent_hispanic
170526,99927,02198000100,0.0,0.0,1.0,1.0,0.019264,0.381786,0.028459
170527,99921,02198000200,0.0,0.0,1.0,1.0,0.009449,0.372835,0.024803
170528,99925,02198000200,0.0,0.0,1.0,1.0,0.009449,0.372835,0.024803
170529,99923,02198000300,0.0,0.0,1.0,1.0,0.000000,0.000000,0.000000
170530,99926,02198940100,0.0,0.0,1.0,1.0,0.006116,0.829969,0.070948


Next we need to look at the residential breakdown of zip codes that span multiple census tracts. If the residential ratios all sum up to 1, we should be able to multiple the demographic percentages by `RES_RATIO` by each row.

In [7]:
zip_total_ratio = hud_ct_zip_df.groupby('ZIP').agg({'RES_RATIO':'sum', 'census_tract':'count'}).reset_index()
zip_total_ratio.columns = ['ZIP', 'total_res_ratio', 'num_census_tracts']
zip_total_ratio.head()

,ZIP,total_res_ratio,num_census_tracts
0,00501,0.0,1
1,01001,1.0,6
2,01002,1.0,10
3,01003,1.0,4
4,01004,1.0,1


However, as we see below, not all total residential ratios ratios are equal to 1; therefore, we cannot just multiply the percentages by the percentage of residential addresses. There are cases where the total residential ratio is 0 (perhaps for non-residential zip codes) and cases where the sum is less than 1 (in rows where ACS demographic data wasn't available and which were therefore dropped in the inner join above).

In [8]:
zip_total_ratio.total_res_ratio.value_counts().iloc[:10]

1.000000    31480
0.000000     4343
1.000000     2499
1.000000      568
1.000000      193
1.000000        3
0.452586        1
0.047573        1
0.982114        1
0.898289        1
Name: total_res_ratio, dtype: int64

For that reason, we will calculate an adjustment factor, so that each zip code's `RES_RATIO` should sum up to 1. From that point on, we should be able to multiply the adjusted ratio by the demographic percentages and sum them up.

In [9]:
zip_total_ratio['ratio_adj_factor'] = 1 / zip_total_ratio['total_res_ratio']
hud_ct_zip_df = pd.merge(hud_ct_zip_df, zip_total_ratio[['ZIP', 'ratio_adj_factor', 'num_census_tracts']], how='inner', on='ZIP')
hud_ct_zip_df.head()

,ZIP,census_tract,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,percent_black,percent_indigenous,percent_hispanic,ratio_adj_factor,num_census_tracts
0,00501,36103158607,0.000000,1.000000,0.000000,1.000000,0.036232,0.000000,0.201087,inf,1
1,11742,36103158607,0.265622,0.431604,0.108844,0.273516,0.036232,0.000000,0.201087,1.0,7
2,11742,36103158605,0.000000,0.030660,0.013605,0.002502,0.037760,0.037760,0.093560,1.0,7
3,11742,36103158606,0.133456,0.075472,0.068027,0.127752,0.011561,0.018411,0.079426,1.0,7
4,11742,36103146611,0.246820,0.268868,0.108844,0.244997,0.135733,0.000000,0.053170,1.0,7


Where the residential ratio sums up to 0 (suggesting no residential addresses in the zip code for the inner-joined data we have), we will divide the demographic weight equally across all census tracts in the zip code.

In [10]:
hud_ct_zip_df['FINAL_RATIO'] = hud_ct_zip_df.RES_RATIO * hud_ct_zip_df.ratio_adj_factor
hud_ct_zip_df['FINAL_RATIO'] = hud_ct_zip_df['FINAL_RATIO'].fillna(1/hud_ct_zip_df['num_census_tracts']).astype(float)
cols = ['percent_black', 'percent_indigenous', 'percent_hispanic']
for col in cols:
    hud_ct_zip_df[col] = hud_ct_zip_df[col] * hud_ct_zip_df['FINAL_RATIO']

In [11]:
# Take the final sum of demographic percentages across census tracts by zip code
zip_demo_df = hud_ct_zip_df.groupby('ZIP')[cols].sum().reset_index()
print(zip_demo_df.shape)
zip_demo_df.head()

(39125, 4)


,ZIP,percent_black,percent_indigenous,percent_hispanic
0,00501,0.036232,0.000000,0.201087
1,01001,0.020084,0.002879,0.052331
2,01002,0.092367,0.013670,0.068443
3,01003,0.056715,0.009107,0.062750
4,01004,0.015564,0.007938,0.030973


The data we save below is what is used by `zipbiaschecker`.

In [12]:
zip_demo_df.to_csv('../data/processed/zipcode_demographic_data.csv', index = False)